# Insight and debug
In order to explore the structure of a solver and to debug eventual mistakes, a few function are provided. Three are methods of the solver class:
- `show_structures()`: print a list of the structures inside the solver
- `show_connections()`: print a list of the internal connections between the structures in the solver
- `show_free_pins()`: print a list of the free pins of the structures inside the solver with the corresponding mapping to the external pins. All the pin showed here needs to me mapped for the code to work.
- `inspect()`: goes through the full hierarchy of the solver, printing for each layer the solvers and models contained in the top solver.

In order to see them in action, some structure are needed. The ones from the guide on parametric blocks will be rebuilt and used.

In [1]:
import lekkersim as lk

with lk.Solver(name='MZM') as MZM_bal:
    WG=lk.Waveguide(L=10.0, n=2.5)
    BM=lk.BeamSplitter(ratio=0.5)
    PS=lk.PhaseShifter(param_name='PS')
    
    bm1=BM.put()
    wg1=WG.put('a0',bm1.pin['b0'])
    wg2=WG.put('a0',bm1.pin['b1'])
    ps1 =PS.put('a0',wg1.pin['b0'],param_mapping={'PS': 'PS1'})
    ps2 =PS.put('a0',wg2.pin['b0'],param_mapping={'PS': 'PS2'})

    bm2=BM.put('a0',ps1.pin['b0'])
    lk.connect(ps2.pin['b0'],bm2.pin['a1'])
    
    lk.Pin('a0').put(bm1.pin['a0'])
    lk.Pin('a1').put(bm1.pin['a1'])
    lk.Pin('b0').put(bm2.pin['b0'])
    lk.Pin('b1').put(bm2.pin['b1'])
    
with lk.Solver(name='IQ_mod') as IQ_mod:
    BM=lk.BeamSplitter(ratio=0.5)
    PS=lk.PhaseShifter(param_name='PS')
    
    bm1=BM.put()
    mz1=MZM_bal.put('a0',bm1.pin['b0'],param_mapping={'PS1' : 'I1', 'PS2' : 'I2'})
    mz2=MZM_bal.put('a0',bm1.pin['b1'],param_mapping={'PS1' : 'Q1', 'PS2' : 'Q2'})
    ps1=PS.put('a0',mz1.pin['b1'], param_mapping={'PS' : 'I'})
    ps2=PS.put('a0',mz2.pin['b1'], param_mapping={'PS' : 'Q'})
    bm2=BM.put('a0',ps1.pin['b0'])
    lk.connect(ps2.pin['b0'],bm2.pin['a1'])
    
    lk.Pin('a0').put(bm1.pin['a0'])
    lk.Pin('a1').put(bm1.pin['a1'])
    lk.Pin('b0').put(bm2.pin['b0'])
    lk.Pin('b1').put(bm2.pin['b1'])
    
    lk.Pin('DumIn0').put(mz1.pin['a1'])
    lk.Pin('DumIn1').put(mz2.pin['a1'])
    lk.Pin('DumOut0').put(mz1.pin['b0'])
    lk.Pin('DumOut1').put(mz2.pin['b0'])

INFO:lekkersim:version 0.0.3.post18+git.d54a189e.dirty


Here is an example of the output on the 'MZM_bal' solver.
### show_structures


In [2]:
MZM_bal.show_structures()

Structures and pins of solver:                 Solver of MZM (id=140035924807760))
  Structure (id=140035056310864) containing Model of beam-splitter with ratio 0.5 (id=140037477956560)
  Structure (id=140035034073424) containing Model of waveguide of lenght 10.000 and index 2.500 (id=140035924840656)
  Structure (id=140035034073936) containing Model of waveguide of lenght 10.000 and index 2.500 (id=140035924840656)
  Structure (id=140035034075088) containing Model object (id=140037477956688) with pins: ['a0', 'b0']
  Structure (id=140035034076624) containing Model object (id=140037477956688) with pins: ['a0', 'b0']
  Structure (id=140035034078096) containing Model of beam-splitter with ratio 0.5 (id=140037477956560)


### show_connections

In [3]:
MZM_bal.show_connections()

Connection of solver:                 Solver of MZM (id=140035924807760))
  (Structure (id=140035034073424) containing Model of waveguide of lenght 10.000 and index 2.500 (id=140035924840656),    a0) <--> (Structure (id=140035056310864) containing Model of beam-splitter with ratio 0.5 (id=140037477956560),    b0)
  (Structure (id=140035034073936) containing Model of waveguide of lenght 10.000 and index 2.500 (id=140035924840656),    a0) <--> (Structure (id=140035056310864) containing Model of beam-splitter with ratio 0.5 (id=140037477956560),    b1)
  (Structure (id=140035034075088) containing Model object (id=140037477956688) with pins: ['a0', 'b0'],    a0) <--> (Structure (id=140035034073424) containing Model of waveguide of lenght 10.000 and index 2.500 (id=140035924840656),    b0)
  (Structure (id=140035034076624) containing Model object (id=140037477956688) with pins: ['a0', 'b0'],    a0) <--> (Structure (id=140035034073936) containing Model of waveguide of lenght 10.000 and index

### show_free_pins

In [4]:
MZM_bal.show_free_pins()

Free pins of solver:                 Solver of MZM (id=140035924807760))
  (Structure (id=140035056310864) containing Model of beam-splitter with ratio 0.5 (id=140037477956560),    a0) -->    a0
  (Structure (id=140035056310864) containing Model of beam-splitter with ratio 0.5 (id=140037477956560),    a1) -->    a1
  (Structure (id=140035034078096) containing Model of beam-splitter with ratio 0.5 (id=140037477956560),    b0) -->    b0
  (Structure (id=140035034078096) containing Model of beam-splitter with ratio 0.5 (id=140037477956560),    b1) -->    b1


## inspect
The `inspect` function could be quite useful to debug complex structures, since it is the only one able to navigate the full hierarchy. For example here is the output on `MZM_bal`:

In [5]:
MZM_bal.inspect()

Solver of MZM (id=140035924807760)
  Structure (id=140035056310864) containing Model of beam-splitter with ratio 0.5 (id=140037477956560)
  Structure (id=140035034073424) containing Model of waveguide of lenght 10.000 and index 2.500 (id=140035924840656)
  Structure (id=140035034073936) containing Model of waveguide of lenght 10.000 and index 2.500 (id=140035924840656)
  Structure (id=140035034075088) containing Model object (id=140037477956688) with pins: ['a0', 'b0']
  Structure (id=140035034076624) containing Model object (id=140037477956688) with pins: ['a0', 'b0']
  Structure (id=140035034078096) containing Model of beam-splitter with ratio 0.5 (id=140037477956560)


In [6]:
IQ_mod.inspect()

Solver of IQ_mod (id=140035034080656)
  Structure (id=140035034082960) containing Model of beam-splitter with ratio 0.5 (id=140035034080144)
  Structure (id=140035034083408) containing Solver of MZM (id=140035924807760)
    Structure (id=140035056310864) containing Model of beam-splitter with ratio 0.5 (id=140037477956560)
    Structure (id=140035034073424) containing Model of waveguide of lenght 10.000 and index 2.500 (id=140035924840656)
    Structure (id=140035034073936) containing Model of waveguide of lenght 10.000 and index 2.500 (id=140035924840656)
    Structure (id=140035034075088) containing Model object (id=140037477956688) with pins: ['a0', 'b0']
    Structure (id=140035034076624) containing Model object (id=140037477956688) with pins: ['a0', 'b0']
    Structure (id=140035034078096) containing Model of beam-splitter with ratio 0.5 (id=140037477956560)
  Structure (id=140035034084624) containing Solver of MZM (id=140035924807760)
    Structure (id=140035056310864) containing